In [1]:
import pandas as pd
import json
from pillaralgos.helpers import data_handler as dh

In [2]:
data = json.load(open("data/sample_lg.json"))

In [3]:
big_df = dh.organize_twitch_chat(data)
print(big_df.iloc[-1,0] - big_df.iloc[0,0])

0 days 10:57:21.037000


In [4]:
def emo_extractor(my_list):
    return [emo_dict['_id'] for emo_dict in my_list]


def loc_extractor(my_list):
    return [[emo_dict['begin'], emo_dict['end']] for emo_dict in my_list]

all_emos = []
def emo_saver(my_list):
    for emo in my_list:
        all_emos.append(emo)
    return 'saved'

In [5]:
emo_col = big_df['emoticons'].dropna().reset_index(drop=True)
emo_id_list = emo_col.apply(lambda my_list: emo_extractor(my_list))
emo_id_list.apply(lambda my_list: emo_saver(my_list))
all_emos = pd.Series(all_emos)

In [6]:
emo_loc = emo_col.apply(lambda my_list: loc_extractor(my_list))
emo_body = big_df[~big_df['emoticons'].isna()]['body']

In [7]:
emo_data = emo_loc.copy().reset_index()
emo_data['body'] = emo_body.reset_index(drop=True)
emo_data = emo_data.drop('index',axis=1)

In [8]:
emo_data['id'] = emo_id_list

In [9]:
emo_dict = {}

In [10]:
for i, row in emo_data.iterrows():
    for x in range(len(row['emoticons'])):
        loc = row['emoticons'][x] # grab location
        begin = loc[0]
        end = loc[1] + 1
        emoji_text = row['body'][begin:end] # extract emoji text
        emoji_id = row['id'][x]
        
        if emoji_id not in emo_dict.keys():
            emo_dict[emoji_id] = emoji_text

In [11]:
num_used = all_emos.value_counts()

In [12]:
num_used = num_used.reset_index()
num_used.columns = ['emoji_id', 'occurrance']

In [13]:
num_used['emoji_text'] = num_used['emoji_id'].map(emo_dict)

In [14]:
num_used['label'] = ''
top_emoticons = num_used[num_used['occurrance'] > num_used['occurrance'].mean()]

top_emoticons.to_csv("data/top_emoticons.csv", index=False)

In [47]:
num_used.describe()

count     510.000000
mean       77.815686
std       477.517172
min         1.000000
25%         1.000000
50%         3.000000
75%        12.000000
max      7709.000000
dtype: float64

In [49]:
num_used[num_used > num_used.mean()].shape

(55,)

# Description

Notebook to explore data, recreate bugs, solve bugs

In [1]:
from pillaralgos import algo2
import pillaralgos_dev as p_dev
from pillaralgos_dev import dev_helpers as dev_help
import json

In [2]:
buggy_file = '981828174'
def bug_handler(buggy_file, algo):
    '''
    Simple function to run reported file and files in data folder through a function
    '''
    aws = p_dev.awsBucketAPI()
    aws.save_specific_file(buggy_file)
    data = json.load(open(f'data/{buggy_file}.json'))
    try:
        json_result = algo.run(data)
    except Exception as e:
        print("FAILED")
        print(e)

    import os
    files = os.listdir('data/')
    new_files = [f for f in files if '.json' in f and 'big_data' not in f]

    algoworked = []
    algofailed = []
    results = {}
    for file in new_files:
        try:
            data = json.load(open(f'data/{file}'))
            results[file] = algo.run(data)
            algoworked.append(file)
        except:
            algofailed.append(file)
    return algoworked, algofailed, results

# Unit Test creations

In [1]:
import json
import pandas as pd

In [3]:
data = json.load(open("pypi/prod/test/sample_data/sample_med.json"))

In [6]:
from pillaralgos.helpers import data_handler as dh

In [39]:
from pillaralgos import algo1, algo2, algo3_0, algo3_5

In [37]:
json_re1 = algo1.run(data,min_=0.5, limit=10, sort_by='rel',save_json=False)

In [40]:
json_re2 = algo2.run(data,min_=0.5, limit=10,save_json=False)

/home/jupyter-pomkos/.conda/envs/pillar_env/lib/python3.8/site-packages/pillaralgos/helpers/data_handler.py:278: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk["hour"] = i
/home/jupyter-pomkos/.conda/envs/pillar_env/lib/python3.8/site-packages/pillaralgos/helpers/data_handler.py:279: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk["chunk"] = x


In [43]:
json_re3 = algo3_0.run(data,min_=0.5, limit=10, min_words=10,save_json=False)

/home/jupyter-pomkos/.conda/envs/pillar_env/lib/python3.8/site-packages/pillaralgos/helpers/data_handler.py:278: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk["hour"] = i
/home/jupyter-pomkos/.conda/envs/pillar_env/lib/python3.8/site-packages/pillaralgos/helpers/data_handler.py:279: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk["chunk"] = x


In [45]:
json_re3 = algo3_5.run(data,min_=0.5, limit=10, goal='num_emo',save_json=False)

/home/jupyter-pomkos/.conda/envs/pillar_env/lib/python3.8/site-packages/pillaralgos/helpers/data_handler.py:278: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk["hour"] = i
/home/jupyter-pomkos/.conda/envs/pillar_env/lib/python3.8/site-packages/pillaralgos/helpers/data_handler.py:279: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk["chunk"] = x


In [46]:
json_re3

[{'startTime': 0.0, 'endTime': 119.562},
 {'startTime': 765.512, 'endTime': 882.645},
 {'startTime': 891.677, 'endTime': 1008.99},
 {'startTime': 259.889, 'endTime': 375.413},
 {'startTime': 127.098, 'endTime': 234.249},
 {'startTime': 383.298, 'endTime': 495.599},
 {'startTime': 630.41, 'endTime': 741.233},
 {'startTime': 1018.379, 'endTime': 1131.467},
 {'startTime': 507.003, 'endTime': 626.884},
 {'startTime': 1138.629, 'endTime': 1256.19}]